In [1]:
! pip install openpyxl pandas numpy

Looking in indexes: https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/simple

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np

channels = pd.read_excel("../data/tg_channels.xlsx")
#df = pd.read_csv("../data/cleaned_news_exp.csv")[["message_id", "id_channel", "message", "date", "topic"]]
df = pd.read_parquet("../data/tg_news_full.parquet")[["message_id", "id_channel", "message", "date"]]

/home/mlcore/conda/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [2]:
ch_map = (channels[["id", "name"]]
          .dropna()
          .assign(id=lambda x: pd.to_numeric(x["id"], errors="coerce"))
          .dropna(subset=["id"])
          .assign(id=lambda x: x["id"].astype(int))
          .set_index("id")["name"]
          .to_dict())

df = df.copy()
df["id_channel"] = pd.to_numeric(df["id_channel"], errors="coerce")
df["channel_name"] = df["id_channel"].map(ch_map).fillna(df["id_channel"].astype("Int64").astype(str))
df.head()

,message_id,id_channel,message,date,channel_name
0,275548,3,"Правительство Словакии обсудит меры, которые м...",2025-01-02 17:00:02,Риа Новости
1,275547,3,В ДТП с туристическим автобусом в Таиланде пос...,2025-01-02 16:40:53,Риа Новости
2,275546,3,Премьер Израиля Нетаньяху выписан из больницы ...,2025-01-02 16:20:12,Риа Новости
3,275545,3,Подозреваемый в подрыве автомобиля Tesla Cyber...,2025-01-02 15:54:29,Риа Новости
4,275543,3,Спецоперация. Обстановка и главные события на ...,2025-01-02 15:32:55,Риа Новости


In [3]:
import re
import numpy as np
import pandas as pd

def clean_news_text(t: str) -> str:
    t = t or ""
    t = re.sub(r"#\w+", " ", t)
    t = re.sub(r"[⚡️📈📉🇷🇺✅❗️🔥⬛ ⬜ ⚫ ⚪🔹]+", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

def ensure_datetime(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    d = df.copy()
    d[col] = pd.to_datetime(d[col], utc=True, errors="coerce")
    d = d.dropna(subset=[col])
    d["date_day"] = d[col].dt.floor("D")
    return d

df = ensure_datetime(df, "date")
df["message_id"] = df["message_id"].astype(str)
df["message"] = df["message"].fillna("").astype(str).map(clean_news_text)

In [4]:
df.head()

,message_id,id_channel,message,date,channel_name,date_day
0,275548,3,"Правительство Словакии обсудит меры, которые м...",2025-01-02 17:00:02+00:00,Риа Новости,2025-01-02 00:00:00+00:00
1,275547,3,В ДТП с туристическим автобусом в Таиланде пос...,2025-01-02 16:40:53+00:00,Риа Новости,2025-01-02 00:00:00+00:00
2,275546,3,Премьер Израиля Нетаньяху выписан из больницы ...,2025-01-02 16:20:12+00:00,Риа Новости,2025-01-02 00:00:00+00:00
3,275545,3,Подозреваемый в подрыве автомобиля Tesla Cyber...,2025-01-02 15:54:29+00:00,Риа Новости,2025-01-02 00:00:00+00:00
4,275543,3,Спецоперация. Обстановка и главные события на ...,2025-01-02 15:32:55+00:00,Риа Новости,2025-01-02 00:00:00+00:00


In [5]:
#! pip install rank_bm25

In [6]:
from rank_bm25 import BM25Okapi

def tokenize_ru(text: str):
    text = text.lower()
    text = re.sub(r"[^0-9a-zа-яё\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

corpus_tok = [tokenize_ru(t) for t in df["message"].tolist()]
bm25 = BM25Okapi(corpus_tok)

In [7]:
#! pip install faiss-cpu faiss-gpu-cu12 faiss-gpu-cu11

In [8]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/multilingual-e5-large" 
encoder = SentenceTransformer(MODEL_NAME, device="cuda")  
texts = df["message"].tolist()
doc_inputs = ["passage: " + t for t in texts]

In [9]:
# E_docs = encoder.encode(
#     doc_inputs,
#     batch_size=64,
#     show_progress_bar=True,
#     normalize_embeddings=True,
# ).astype(np.float32)

In [10]:
# import faiss

# dim = E_docs.shape[1]
# index = faiss.IndexFlatIP(dim)      
# index.add(E_docs)

In [11]:
# # save
# from pathlib import Path
# import numpy as np
# import pandas as pd
# import faiss
# import pickle

# OUT = Path("indexes")
# OUT.mkdir(parents=True, exist_ok=True)

# #rowmap
# rowmap = df[["message_id","date","date_day","id_channel","channel_name"]].copy()
# rowmap.to_parquet(OUT / "rowmap.parquet", index=False)

# np.save(OUT / "E_docs_e5_large.npy", E_docs)

# faiss.write_index(index, str(OUT / "faiss_e5_large.index"))

# with open(OUT / "bm25_corpus_tok.pkl", "wb") as f:
#     pickle.dump(corpus_tok, f)


In [12]:
# load
from pathlib import Path
import numpy as np
import pandas as pd
import faiss
import pickle

INP = Path("indexes")

rowmap = pd.read_parquet(INP / "rowmap.parquet")

E_docs = np.load(INP / "E_docs_e5_large.npy")
index = faiss.read_index(str(INP / "faiss_e5_large.index"))

with open(INP / "bm25_corpus_tok.pkl", "rb") as f:
    corpus_tok = pickle.load(f)

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(corpus_tok)


In [13]:
assert len(rowmap) == E_docs.shape[0]
assert len(rowmap) == index.ntotal

In [14]:
import numpy as np
import pandas as pd
import re

def snippet(t: str, n: int = 1000) -> str:
    return t[:n]

def _topk_indices_from_scores(scores: np.ndarray, k: int) -> np.ndarray:
    k = min(k, len(scores))
    if k <= 0:
        return np.array([], dtype=int)
    if k == len(scores):
        idx = np.argsort(-scores)
    else:
        idx = np.argpartition(-scores, k-1)[:k]
        idx = idx[np.argsort(-scores[idx])]
    return idx.astype(int)

def dense_candidates_faiss(index, encoder, query: str, topN: int = 500):
    qv = encoder.encode(["query: " + query], normalize_embeddings=True, show_progress_bar=False).astype(np.float32)
    scores, idx = index.search(qv, topN) 
    return idx[0].astype(int), scores[0].astype(np.float32)


def hybrid_retrieve_rrf(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    k: int = 50,
    topN_each: int = 500,
    k_rrf: int = 60,
    w_dense: float = 1.0,
    w_bm25: float = 1.0,
    exclude_message_id: str | None = None,
    anchor_date: str | pd.Timestamp | None = None,
    date_col: str = "date_day") -> pd.DataFrame:

    if anchor_date is not None:
        ad = pd.to_datetime(anchor_date, utc=True).normalize()

        if date_col not in df.columns:
            raise KeyError(f"date_col='{date_col}' not found in df.columns")

        dts = pd.to_datetime(df[date_col], errors="coerce", utc=True).dt.normalize()
        allowed = (dts <= ad)
        allowed_np = allowed.to_numpy(dtype=bool)
    else:
        allowed_np = None

    d_idx, d_sc = dense_candidates_faiss(index, encoder, query, topN=topN_each)

    if allowed_np is not None:
        keep = allowed_np[d_idx]
        d_idx = d_idx[keep]
        d_sc = d_sc[keep]

    dense_rank = {int(rowpos): r for r, rowpos in enumerate(d_idx, start=1)}

    if bm25 is None:
        out = df.iloc[d_idx].copy()
        out["_rowpos"] = d_idx
        out["score_rrf"] = (w_dense / (k_rrf + out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan)).astype(np.float32)))
        out["rank_dense"] = out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan))
        out["rank_bm25"] = np.nan

        if exclude_message_id is not None and "message_id" in out.columns:
            out = out[out["message_id"].astype(str) != str(exclude_message_id)]

        return out.head(k).reset_index(drop=True)

    bm_scores = bm25.get_scores(tokenize_fn(query)).astype(np.float32)
    if allowed_np is not None:
        bm_scores[~allowed_np] = -np.inf

    b_idx = _topk_indices_from_scores(bm_scores, topN_each)
    bm_rank = {int(rowpos): r for r, rowpos in enumerate(b_idx, start=1)}

    union = np.array(sorted(set(dense_rank) | set(bm_rank)), dtype=int)
    rrf = np.zeros(len(union), dtype=np.float32)

    for j, rowpos in enumerate(union):
        if rowpos in dense_rank:
            rrf[j] += w_dense / (k_rrf + dense_rank[rowpos])
        if rowpos in bm_rank:
            rrf[j] += w_bm25 / (k_rrf + bm_rank[rowpos])

    order = np.argsort(-rrf)
    union = union[order]
    rrf = rrf[order]

    out = df.iloc[union].copy()
    out["_rowpos"] = union
    out["score_rrf"] = rrf
    out["rank_dense"] = out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan))
    out["rank_bm25"] = out["_rowpos"].map(lambda rp: bm_rank.get(int(rp), np.nan))

    if exclude_message_id is not None and "message_id" in out.columns:
        out = out[out["message_id"].astype(str) != str(exclude_message_id)]

    return out.head(k).reset_index(drop=True)



In [15]:
df["date_day"].min(), df["date_day"].max()

(Timestamp('2023-09-23 00:00:00+0000', tz='UTC'),
 Timestamp('2025-09-08 00:00:00+0000', tz='UTC'))

In [16]:
# ! pip install transformers torch sentence-transformers accelerate vllm

In [17]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# MODEL = "Qwen/Qwen2.5-14B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL,
#     torch_dtype=torch.bfloat16,
#     device_map="cuda",
#     trust_remote_code=True,
# )
# model.eval()

In [19]:
from transformers import AutoTokenizer, AutoConfig
from vllm import LLM, SamplingParams
import torch

MODEL = "Qwen/Qwen2.5-32B-Instruct"
#MODEL = "mistralai/Mistral-Small-3.1-24B-Instruct-2503"

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

cfg = AutoConfig.from_pretrained(MODEL, trust_remote_code=True)
native_ctx = getattr(cfg, "max_position_embeddings", None)
print("GPU:", torch.cuda.get_device_name(0))
print("VRAM(GB):", round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 1))
print("Model max_position_embeddings:", native_ctx)

MAX_MODEL_LEN = 24576


model = LLM(
    model=MODEL,
    dtype="bfloat16",
    max_model_len=MAX_MODEL_LEN,
    gpu_memory_utilization=0.90,
)

GPU: NVIDIA A100-SXM4-80GB
VRAM(GB): 79.3
Model max_position_embeddings: 32768
INFO 01-09 02:27:33 [utils.py:253] non-default args: {'dtype': 'bfloat16', 'max_model_len': 24576, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-32B-Instruct'}
INFO 01-09 02:27:34 [model.py:514] Resolved architecture: Qwen2ForCausalLM
INFO 01-09 02:27:34 [model.py:1661] Using max model len 24576
INFO 01-09 02:27:34 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=9577) INFO 01-09 02:27:41 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='Qwen/Qwen2.5-32B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-32B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=24576, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=N

(EngineCore_DP0 pid=9577) /home/mlcore/conda/lib/python3.10/site-packages/tvm_ffi/_optional_torch_c_dlpack.py:174: UserWarning: Failed to JIT torch c dlpack extension, EnvTensorAllocator will not be enabled.
(EngineCore_DP0 pid=9577) We recommend installing via `pip install torch-c-dlpack-ext`
(EngineCore_DP0 pid=9577)   warnings.warn(


(EngineCore_DP0 pid=9577) INFO 01-09 02:27:46 [cuda.py:351] Using FLASH_ATTN attention backend out of potential backends: ('FLASH_ATTN', 'FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION')


Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   6% Completed | 1/17 [00:00<00:11,  1.44it/s]
Loading safetensors checkpoint shards:  12% Completed | 2/17 [00:01<00:11,  1.31it/s]
Loading safetensors checkpoint shards:  18% Completed | 3/17 [00:02<00:10,  1.29it/s]
Loading safetensors checkpoint shards:  24% Completed | 4/17 [00:03<00:09,  1.31it/s]
Loading safetensors checkpoint shards:  29% Completed | 5/17 [00:03<00:09,  1.33it/s]
Loading safetensors checkpoint shards:  35% Completed | 6/17 [00:04<00:08,  1.34it/s]
Loading safetensors checkpoint shards:  41% Completed | 7/17 [00:05<00:07,  1.28it/s]
Loading safetensors checkpoint shards:  47% Completed | 8/17 [00:05<00:06,  1.37it/s]
Loading safetensors checkpoint shards:  53% Completed | 9/17 [00:06<00:05,  1.38it/s]
Loading safetensors checkpoint shards:  59% Completed | 10/17 [00:07<00:05,  1.34it/s]
Loading safetensors checkpoint shards:  65% Completed | 11/17

(EngineCore_DP0 pid=9577) INFO 01-09 02:28:01 [default_loader.py:308] Loading weights took 13.35 seconds
(EngineCore_DP0 pid=9577) INFO 01-09 02:28:01 [gpu_model_runner.py:3659] Model loading took 61.0375 GiB memory and 17.955664 seconds
(EngineCore_DP0 pid=9577) INFO 01-09 02:28:14 [backends.py:643] Using cache directory: /home/mlcore/.cache/vllm/torch_compile_cache/956b349265/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=9577) INFO 01-09 02:28:14 [backends.py:703] Dynamo bytecode transform time: 12.92 s
(EngineCore_DP0 pid=9577) INFO 01-09 02:28:35 [backends.py:226] Directly load the compiled graph(s) for compile range (1, 8192) from the cache, took 14.025 s
(EngineCore_DP0 pid=9577) INFO 01-09 02:28:35 [monitor.py:34] torch.compile takes 26.95 s in total
(EngineCore_DP0 pid=9577) INFO 01-09 02:28:36 [gpu_worker.py:375] Available KV cache memory: 8.81 GiB
(EngineCore_DP0 pid=9577) INFO 01-09 02:28:37 [kv_cache_utils.py:1291] GPU KV cache size: 36,064 tokens
(EngineCo

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:06<00:00,  7.50it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:03<00:00,  9.78it/s]


(EngineCore_DP0 pid=9577) INFO 01-09 02:28:48 [gpu_model_runner.py:4587] Graph capturing finished in 11 secs, took 4.11 GiB
(EngineCore_DP0 pid=9577) INFO 01-09 02:28:48 [core.py:259] init engine (profile, create kv cache, warmup model) took 46.87 seconds
INFO 01-09 02:28:50 [llm.py:360] Supported tasks: ['generate']
ERROR 01-09 04:44:27 [core_client.py:606] Engine core proc EngineCore_DP0 died unexpectedly, shutting down client.


In [71]:
SYSTEM_PROMPT = """Ты — очень внимательный аналитик новостного потока (экономика/финансы, но запрос может быть любым).

Вход: запрос (тема/новость), опорная дата (YYYY-MM-DD) и набор документов:
[i] date=YYYY-MM-DD channel=<название> id=<...>
<текст>

Сформируй читабельный аналитический отчёт строго в 4 блоках:

1) Запрос и дата
2) Дайджест (расширенное связное саммари, включая различия в подаче каналов)
3) Выводы и актуальность
4) Таймлайн (полный, от нового к старому; одна строка = одна дата)

========================
КРИТИЧЕСКИЕ ПРАВИЛА
===================

1) Пиши строго на русском языке.
2) Используй ТОЛЬКО информацию из предоставленных документов. Никаких внешних фактов, общих знаний и “контекста дня”.
3) Никогда не упоминай номера [i] и не делай ссылок вида [1][2].
4) Любая конкретика (числа, уровни, оценки, причины, “почему”, выводы) — только с привязкой к источнику прямо в тексте:
   “Канал (YYYY-MM-DD): …”
   Можно добавлять id, если он дан: “Канал (YYYY-MM-DD, id=...): …”
5) Не выдумывай причин/последствий. Если в источниках нет объяснений — явно напиши “в источниках причины не поясняются”.
6) Не используй слово “anchor_date”. Пиши “актуальная дата”.
7) Если документов рядом с опорной датой мало/нет — НЕ додумывай “что происходит сейчас”. Вместо этого:
   • честно зафиксируй разрыв во времени,
   • опиши, что именно можно сказать по тем документам, которые есть,
   • отметь, каких данных не хватает для уверенного вывода о текущем состоянии темы.

========================
ЖЕСТКИЕ ПРАВИЛА ДЛЯ ИЗБЕЖАНИЯ ОШИБОК
====================================

A) “Атомарность факта” (ключевое правило против склеек)

Любая фраза, где есть конкретное значение (число/процент/уровень/дата события/формальная формулировка), должна быть атомарной:
• внутри фразы допускается ТОЛЬКО ОДНА ссылка вида “Канал (YYYY-MM-DD)”;
• и эта фраза должна содержать только то значение/формулировку, которое относится к этой дате.

Запрещено:
• переносить число/оценку на другой день “по смыслу”;
• склеивать разные значения в одну фразу;
• писать “X (дата1; дата2)” если X подтверждено не на обе даты.

B) Обобщения — только после опорных точек
Если делаешь вывод уровня “вырос/снизился/обострилось/усилилось/ослабло/вернулось/держалось”, то:
• сначала перечисли минимум 2–3 опорные точки с датами и каналами (в тексте),
• и только затем сформулируй обобщение.
Если точек мало — используй нейтральное: “встречаются эпизоды”, “динамика неоднородна”, “фактов недостаточно”.

C) Запрет “сильных трендов” без достаточных точек
Слова “тренд / устойчиво / продолжает / ускоряется / разворачивается” разрешены только если:
• есть минимум 3 релевантные точки по времени,
• и они согласованно подтверждают направление.
Иначе: “были эпизоды”, “наблюдаются всплески”, “картина смешанная”.

D) Причины/объяснения — только как позиция источника
Запрещено писать “из-за X” или “привело к Y”, если источник не формулирует связь.
Разрешено только так:
• “Канал (дата) связывает X с Y / объясняет … / передаёт мнение …”
Если причины расходятся — отметить “в источниках встречаются разные объяснения”.

E) Тип/контекст факта не угадывать
Если источник не уточняет, “что именно это” (тип показателя, метод подсчёта, статус события, юридический контекст и т.п.):
• повторяй формулировку источника или добавь “контекст/тип не уточнён”,
• НЕ подставляй уточнения по умолчанию.

F) Таймлайн без интерпретаций
• одна строка = одна дата;
• если в одну дату несколько каналов — объединяй через “;”;
• в таймлайне только “что произошло/что утверждается”, без причин и выводов.

G) Валидность дат
Запрещено добавлять даты/события, которых нет в предоставленных документах.
Таймлайн должен содержать только даты, присутствующие в контексте.

========================
ОСОБЫЙ РЕЖИМ “КОНТЕКСТ ПУСТОЙ”
==============================

Если в предоставленных документах нет ни одной релевантной новости по теме (или список источников пуст / все источники нерелевантны запросу),
то напиши ТОЛЬКО 3 блока:

### 1) Запрос и дата
### 2) Дайджест
### 3) Выводы и актуальность

В начале блоков 2 и 3 обязательно добавь:
“В данных нет новостей по теме, ниже — общий комментарий без опоры на источники”.

Контекст считается пустым, если выполнено хотя бы одно:
• после заголовка ИСТОЧНИКИ нет ни одного документа;
• или все документы нерелевантны запросу.

========================
ОГРАНИЧЕНИЕ ОБЪЁМА
==================

• Дайджест: 4–10 абзацев (можно более развернуто и литературно, но без потери точности).
• Выводы: 2–6 буллетов.
• Таймлайн: полный по всем датам из контекста (без лимита по числу строк).

========================
ТРЕБОВАНИЯ К СОДЕРЖАНИЮ БЛОКОВ
==============================

### 1) Запрос и дата
* Запрос: ...
* Опорная дата: ...

### 2) Дайджест (расширенный, “красивый” язык без потери строгости)
Требования:
• Начни с “за последнее время” (самые близкие к актуальной дате документы): 2–5 ключевых фактов с датами и каналами.
• Затем опиши предысторию и “волны/разрывы” (если они видны).
• Встрой в текст мини-срез “как писали каналы” (без отдельного блока):
  - какие каналы что подчеркивают/какую рамку выбирают/какую лексику используют (нейтрально),
  - 2–4 опорные точки по каждому заметному каналу (даты в скобках допустимы для “упоминал/писал”, но не для одного числа).
• Если рядом с опорной датой нет материалов — явно напиши, что по данным сделать вывод о текущем положении нельзя.

### 3) Выводы и актуальность
2–6 буллетов, фокус на наблюдаемых сигналах:
* свежесть (дата последнего упоминания и разрыв до опорной даты, если можно посчитать)
* плотность (волна или единичные упоминания)
* ширина (сколько каналов)
* концентрация (доминирует ли один)
* новостность (похоже на разовое событие или на регулярный мониторинг/серии публикаций)
* ограничения данных (разрывы/шум/чего не хватает)
Запрещено: “важно/значимо” без опоры на сигналы выше.

### 4) Таймлайн
* Список строк, от нового к старому:
  “YYYY-MM-DD — Канал1; Канал2: кратко что произошло (по формулировке источника)”
* Если объединяешь каналы на одной дате — формулировка должна быть совместимой для всех перечисленных каналов.
  Если у каналов на одной дате разные конкретные значения/утверждения — перечисляй их через “;” в рамках одной строки, не усредняя и не подменяя.

========================
ФОРМАТ ВЫВОДА (СТРОГО)
======================

### 1) Запрос и дата
...

### 2) Дайджест
...

### 3) Выводы и актуальность
* ...
* ...

### 4) Таймлайн
* YYYY-MM-DD — ...
* ...
"""



def build_rag_context(query: str, cand: pd.DataFrame, anchor_date: str, k_docs: int = 30, snip_chars: int = 850) -> str:
    c = cand.head(k_docs).copy()

    blocks = []
    for i, row in enumerate(c.itertuples(index=False), start=1):
        date_day = getattr(row, "date_day", getattr(row, "date", ""))
        if isinstance(date_day, pd.Timestamp):
            date_day = date_day.strftime("%Y-%m-%d")
        date_day = str(date_day)[:10]

        channel = getattr(row, "channel_name")
        text = getattr(row, "message", "")

        blocks.append(f"[{i}] date={date_day} channel={channel}\n" + snippet(str(text), snip_chars))

    return (
        f"anchor_date: {anchor_date}\n"
        f"ВОПРОС/ЗАПРОС:\n{query}\n\n"
        f"ИСТОЧНИКИ:\n" + "\n\n".join(blocks)
    )



In [72]:
import re, json
import pandas as pd
from vllm import SamplingParams

JUDGE_SYSTEM = """Ты — строгий эксперт по информационному поиску по новостям (в т.ч. экономическим).

Твоя задача: оценить релевантность кандидатной новости запросу. Запрос может быть:
- коротким топиком (например "курс рубля к доллару"),
- или текстом другой новости (тогда запрос описывает конкретный инфоповод).

Используй ТОЛЬКО текст кандидатного документа. Ничего не додумывай.

Шкала релевантности:
2 — документ явно про то же самое: отвечает топику ИЛИ описывает тот же инфоповод/факт/событие, что и запрос.
1 — документ связан по теме/контексту, но это немного другой инфоповод, или про то же, но без прямого соответствия.
0 — нерелевантно совсем.

Правило строгости:
ставь 2 только если связь очевидна по тексту документа; если информации недостаточно — ставь 0 или 1.

Верни строго валидный JSON и ничего больше:
{"relevance": 0|1|2}
"""


def _parse_relevance(text: str) -> int:
    text = text.strip()
    m = re.search(r"\{.*\}", text, flags=re.DOTALL)
    if m:
        blob = m.group(0)
        try:
            obj = json.loads(blob)
            val = int(obj.get("relevance", 0))
            return val if val in (0, 1, 2) else 0
        except Exception:
            pass
    m2 = re.search(r"relevance\"\s*:\s*([012])", text)
    if m2:
        return int(m2.group(1))
    return 0

def judge_filter_candidates(
    cand: pd.DataFrame,
    query: str,
    judge_llm,
    judge_tokenizer,
    *,
    keep_threshold: int = 1,     
    doc_max_chars: int = 1200,
    batch_size: int = 32,
    max_out_tokens: int = 40,
) -> pd.DataFrame:
    if cand is None or len(cand) == 0:
        return cand

    text_col = "message"
    channel_col = "channel_name"
    date_col = "date_day"

    prompts = []
    for _, row in cand.iterrows():
        doc = str(row[text_col])[:doc_max_chars]
        ch = str(row[channel_col]) if channel_col else ""
        dt = str(row[date_col]) if date_col else ""

        user_msg = (
            f"ЗАПРОС:\n{query}\n\n"
            f"КАНДИДАТ:\n"
            f"channel={ch}\n"
            f"date={dt}\n"
            f"text:\n{doc}\n"
        )

        messages = [
            {"role": "system", "content": JUDGE_SYSTEM},
            {"role": "user", "content": user_msg},
        ]
        prompt = judge_tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        prompts.append(prompt)

    sampling = SamplingParams(
        temperature=0.0,
        top_p=1.0,
        max_tokens=max_out_tokens,
    )

    relevances = []
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i:i+batch_size]
        outs = judge_llm.generate(batch_prompts, sampling)
        for o in outs:
            txt = o.outputs[0].text
            relevances.append(_parse_relevance(txt))

    out_df = cand.copy()
    out_df["judge_relevance"] = relevances

    filtered = out_df[out_df["judge_relevance"] >= keep_threshold].copy()
    filtered.reset_index(drop=True, inplace=True)
    return filtered


In [73]:
# import torch

# @torch.inference_mode()
# def rag_summarize(sum_model, sum_tokenizer, query: str, cand: pd.DataFrame, anchor_date, 
#                   k_docs: int = 25, snip_chars: int = 900, max_new_tokens: int = 2000) -> str:
    
#     user = build_rag_context(query, cand, anchor_date=anchor_date, k_docs=k_docs, snip_chars=snip_chars)
#     print("built context...")
    
#     messages = [
#         {"role": "system", "content": SYSTEM_PROMPT},
#         {"role": "user", "content": user},
#     ]
#     prompt = sum_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     enc = sum_tokenizer(prompt, return_tensors="pt", truncation=True).to(sum_model.device)

#     out_ids = sum_model.generate(
#         **enc,
#         max_new_tokens=max_new_tokens,
#         do_sample=False,
#         eos_token_id=sum_tokenizer.eos_token_id,
#         pad_token_id=sum_tokenizer.eos_token_id,
#     )
#     prompt_len = int(enc["attention_mask"][0].sum().item())
    
#     return sum_tokenizer.decode(out_ids[0][prompt_len:], skip_special_tokens=True).strip(), user


In [74]:
from vllm import SamplingParams

def rag_summarize(sum_model, sum_tokenizer, query: str, cand: pd.DataFrame, anchor_date,
                  k_docs: int = 25, snip_chars: int = 900, max_new_tokens: int = 2000):

    user = build_rag_context(query, cand, anchor_date=anchor_date, k_docs=k_docs, snip_chars=snip_chars)
    print("built context...")

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user},
    ]

    prompt = sum_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    #Mistral
    #prompt = f"<s>[SYSTEM_PROMPT]{SYSTEM_PROMPT}[/SYSTEM_PROMPT][INST]{user}[/INST]"

    sampling = SamplingParams(
        temperature=0.0,  
        max_tokens=max_new_tokens,
        top_p=1.0,
    )

    result = sum_model.generate([prompt], sampling)[0]
    text = result.outputs[0].text.strip()

    return text, user


In [75]:
def run_rag_hybrid(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    exclude_message_id: str | None = None,
    k_retrieve: int = 50,
    topN_each: int = 500,
    k_docs: int = 25,
    snip_chars: int = 1500,
    max_new_tokens: int = 2000,
    anchor_date: str = "2025-09-04",
    
    sum_model=None,
    sum_tokenizer=None,

    judge_llm=None,
    judge_tokenizer=None,
    judge_keep_threshold: int = 1,
    judge_batch_size: int = 32,
):
    cand = hybrid_retrieve_rrf(
        df=df,
        index=index,
        encoder=encoder,
        bm25=bm25,
        tokenize_fn=tokenize_fn,
        query=query,
        k=k_retrieve,
        topN_each=topN_each,
        k_rrf=60,
        w_dense=1.0,
        w_bm25=1.0,
        exclude_message_id=exclude_message_id,
        anchor_date=anchor_date
    )

    cand_before = cand
    if judge_llm is not None and judge_tokenizer is not None and cand is not None and len(cand) > 0:
        cand = judge_filter_candidates(
            cand=cand,
            query=query,
            judge_llm=judge_llm,
            judge_tokenizer=judge_tokenizer,
            keep_threshold=judge_keep_threshold,
            doc_max_chars=snip_chars,
            batch_size=judge_batch_size,
        )

    if sum_model is None or sum_tokenizer is None:
        ctx = build_rag_context(query, cand, anchor_date=anchor_date, k_docs=k_docs, snip_chars=snip_chars)
        return {
            "context": ctx,
            "candidates": cand_before,
            "candidates_filtered": cand,
            "summary": "No LLM",
        }

    summary, ctx = rag_summarize(
        sum_model, sum_tokenizer,
        query, cand,
        k_docs=min(k_docs, len(cand)) if cand is not None else 0,
        snip_chars=snip_chars,
        max_new_tokens=max_new_tokens,
        anchor_date=anchor_date
    )

    return {
        "context": ctx,
        "summary": summary,
        "candidates": cand_before,
        "candidates_filtered": cand,
    }


In [76]:
q = "Кофе дорожает"

out = run_rag_hybrid(
    df=df,
    index=index,
    encoder=encoder,
    bm25=bm25,
    tokenize_fn=tokenize_ru,
    query=q,

    k_retrieve=150,
    topN_each=2000,
    k_docs=50,
    snip_chars=1500,
    max_new_tokens=5000,
    anchor_date="2025-09-04",

    sum_model=model,
    sum_tokenizer=tokenizer,

    judge_llm=model,
    judge_tokenizer=tokenizer,
    judge_keep_threshold=1,  
    judge_batch_size=32)


Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

...

Adding requests:   0%|          | 0/22 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/22 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

built context...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [77]:
from IPython.display import display, Markdown, HTML
import re

def show_summary(summary: str):
    if summary is None:
        display(HTML("<b>summary is None</b>"))
        return

    s = str(summary)
    s = s.replace("\\n", "\n") 
    s = re.sub(r"\n{3,}", "\n\n", s).strip()

    display(Markdown(s))

show_summary(out["summary"])


### 1) Запрос и дата
* Запрос: Кофе дорожает
* Опорная дата: 2025-09-04

### 2) Дайджест
За последнее время, ближе к актуальной дате, источники сообщают о продолжении роста цен на кофе. В частности, Блумберг (2025-08-30) отмечает, что сорта кофе робуста и арабика стали самыми подорожавшими товарами в августе, с ростом цен на 1,5 и 29% соответственно. Банки, деньги, два офшора (2025-08-01) предупреждают о новом подорожании кофе на 20–40% из-за проблем с урожаем в Бразилии и Вьетнаме. 

В предыстории, начиная с конца 2024 года, источники фиксировали рост цен на кофе, связанный с неблагоприятными погодными условиями в странах-производителях. Forbes Russia (2024-12-13) и Банки, деньги, два офшора (2025-03-24) предупреждали о возможном росте цен на 40% из-за засухи в Бразилии и проливных дождей во Вьетнаме. Блумберг (2024-10-01) отмечал удвоение цен на зерновой кофе за последний год, а также дальнейший рост из-за засухи и сокращения урожайности.

Каналы подчеркивают разные аспекты проблемы. Банки, деньги, два офшора (2025-03-07) и Forbes Russia (2025-04-07) акцентируют внимание на глобальном дефиците кофе и его влиянии на российский рынок. Блумберг (2025-04-07) и Риа Новости (2025-02-09) отмечают рост цен на растворимый кофе и воду для заваривания кофе. 

### 3) Выводы и актуальность
* Свежесть: Последние упоминания о росте цен на кофе датируются 2025-08-30, что позволяет сделать вывод о текущем положении.
* Плотность: Наблюдается волна упоминаний о росте цен на кофе, особенно в 2025 году.
* Ширина: Множество каналов (Банки, деньги, два офшора, Forbes Russia, Блумберг, Риа Новости) освещают тему.
* Концентрация: Основные каналы (Банки, деньги, два офшора и Блумберг) доминируют в освещении темы.
* Новостность: Тема рассматривается как регулярный мониторинг, а не разовое событие.
* Ограничения данных: Нет данных за последние две недели, что ограничивает возможность делать точные выводы о текущем положении.

### 4) Таймлайн
* 2025-08-30 — Раньше всех. Ну почти: сорта кофе робуста и арабика стали самыми подорожавшими товарами в августе, с ростом цен на 1,5 и 29% соответственно.
* 2025-08-01 — Блумберг: ожидается новое подорожание кофе на 20–40% из-за проблем с урожаем в Бразилии и Вьетнаме.
* 2025-07-18 — Forbes Russia: розничные продажи растворимого кофе сократились почти на 5%, но совокупные продажи кофе увеличились на 1,3%.
* 2025-07-10 — Банки, деньги, два офшора: кофе подорожает на 40% в этом году из-за экстремальной погоды в странах-производителях.
* 2025-06-16 — Риа Новости: кофе и какао могут подорожать из-за торговых войн.
* 2025-05-14 — Раньше всех. Ну почти: с 20 мая запустят торги фьючерсами на кофе.
* 2025-05-14 — Forbes Russia: торги фьючерсами на кофе на срочном рынке Мосбиржи начнутся с 20 мая.
* 2025-04-19 — Риа Новости: колумбийский эксперт Себастьян Сулуага предсказывает продолжение роста цен на кофе.
* 2025-04-07 — Блумберг: кофе в России может подорожать на 50% из-за засухи в Бразилии и роста логистических расходов.
* 2025-04-07 — Банки, деньги, два офшора: кофе в России подорожает на 50% из-за засухи в Бразилии и роста логистических расходов.
* 2025-03-17 — Блумберг: чай подорожает в 2025 году из-за роста стоимости сырья, упаковки, логистики и увеличения спроса на фоне подорожания кофе.
* 2025-03-07 — Forbes Russia: мировая торговля кофе практически остановилась из-за резкого роста цен.
* 2025-03-07 — Раньше всех. Ну почти: мировая торговля кофе практически остановилась из-за резкого роста цен.
* 2025-03-04 — Банки, деньги, два офшора: кофе подорожает на 30-40% из-за усложнения логистики, санкций, роста цен на рабочую силу и топливо, а также изменений климата.
* 2025-02-15 — Банки, деньги, два офшора: в московской Кофемании начали продавать воду для заваривания кофе за 1000 рублей.
* 2025-02-15 — Экономика: в московской Кофемании начали продавать воду для заваривания кофе за 1000 рублей.
* 2025-02-09 — Банки, деньги, два офшора: растворимый кофе в магазинах начали продавать за 1,5 тысячи рублей.
* 2025-01-30 — Блумберг: цены на кофе в России подскочат ещё на 20% уже весной.
* 2025-01-30 — Forbes Russia: крупнейшие поставщики собираются увеличить отпускные цены на кофе в среднем на 20% с марта.
* 2025-01-29 — Риа Новости: биржевые цены на кофе сорта арабика вновь бьют рекорды, поднявшись выше $8 тыс за тонну.
* 2025-01-27 — Риа Новости: биржевые цены на арабику достигли нового рекорда – более $7,8 тыс за тонну.
* 2025-01-10 — Блумберг: Россияне всё чаще отказываются от кофе навынос из-за высоких цен и переходят на домашнее приготовление.
* 2025-01-09 — Экономика: цены на кофе и шоколад в этом году взлетят на 25%.
* 2025-01-07 — Банки, деньги, два офшора: девушка съездила на курорт Архыз и отдала 2400 рублей за два стаканчика кофе.
* 2024-12-31 — Банки, деньги, два офшора: кофе в России станет дороже ещё на 25% из-за засухи в Бразилии и тайфунов во Вьетнаме.
* 2024-12-23 — Банки, деньги, два офшора: кофе в России крайне сильно подорожает в 2025 году из-за улетевшей в космос арабики.
* 2024-12-13 — Forbes Russia: цены на кофе продолжают расти и после достижения рекордных значений, на их снижение могут потребоваться годы.
* 2024-12-10 — Раньше всех. Ну почти: кофе стал одним из самых востребованных товаров года из-за возможного глобального дефицита.
* 2024-12-04 — Экономика: фрукты, чай и кофе из-за рубежа взлетят в цене на 10–20% уже в этом и следующем месяцах.
* 2024-12-03 — Банки, деньги, два офшора: Россияне переходят на кофе «3 в 1», за год его продажи выросли на 10%.
* 2024-12-03 — Блумберг: Россияне переходят на кофе «3 в 1», за год его продажи выросли на 10%.
* 2024-12-01 — Банки, деньги, два офшора: кофе в России станет дороже ещё на 25% из-за засухи в Бразилии и тайфунов во Вьетнаме.
* 2024-12-01 — Экономика: кофе в России в следующем году может резко подорожать на 25%.
* 2024-12-01 — Блумберг: в следующем году кофе в России может подорожать на 25%.
* 2024-11-27 — Блумберг: стоимость кофе сорта арабика продолжает расти на фоне опасений за неурожай в Бразилии.
* 2024-11-27 — Раньше всех. Ну почти: биржевые цены на кофе сорта арабика в среду продолжают бить рекорды, поднявшись до самых высоких уровней с 1997 года.
* 2024-11-25 — Раньше всех. Ну почти: биржевые цены на кофе сорта арабика в понедельник находятся вблизи новых рекордов с 2011 года.
* 2024-11-27 — Раньше всех. Ну почти: биржевые цены на кофе сорта арабика в среду продолжают бить рекорды, поднявшись до самых высоких уровней с 1997 года.
* 2024-10-31 — Сигналы РЦБ: продажи кофе на вынос в России за год увеличились на четверть в денежном выражении.
* 2024-10-31 — Банки, деньги, два офшора: капучино и латте резко подорожают из-за роста цен на зерно и молоко.
* 2024-10-24 — Сигналы РЦБ: продажи кофе на вынос выросли на 25% за год.
* 2024-10-24 — Сигналы РЦБ: количество покупок кофе с собой за 9мес2024 выросло на 14% г/г.
* 2024-10-01 — Блумберг: цены на зерновой кофе удвоились и продолжат расти из-за засухи и сокращения урожайности в Бразилии и Вьетнаме.
* 2024-09-26 — Банки, деньги, два офшора: кофе в России резко подорожает до 30% к концу года из-за засухи в Бразилии и Вьетнаме.
* 2024-09-16 — Банки, деньги, два офшора: цены на кофе взлетели до максимума за 13 лет из-за снижения производства кофе в Бразилии.
* 2024-07-16 — Экономика: семьям погибших на СВО предложили продавать кофе.
* 2024-07-10 — Банки, деньги, два офшора: кофе подорожает на 40% в этом году из-за экстремальной погоды в странах-производителях.

In [78]:
out["context"]

'anchor_date: 2025-09-04\nВОПРОС/ЗАПРОС:\nКофе дорожает\n\nИСТОЧНИКИ:\n[1] date=2025-07-10 channel=Банки, деньги, два офшора\nКофе подорожает на 40% в этом году — прогноз экспертов. Цены вырастут из-за экстремальной погоды в странах-производителях. В Бразилии из-за засухи, во Вьетнаме — проливных дождей. Вскоре одна чашка кофе будет стоить минимум 500 рублей. @bankrollo\n\n[2] date=2025-03-24 channel=Банки, деньги, два офшора\nКофе может резко подорожать до 40% уже в этом году, предупреждают участники рынка. Сильнее всего подорожают сорта арабика и органический кофе. Это произойдёт из-за засухи в Бразилии и роста логистических издержек. @bankrollo\n\n[3] date=2024-12-13 channel=Forbes Russia\nЦены на кофе продолжают расти и после достижения рекордных значений, на их снижение могут потребоваться годы, прогнозируют аналитики. На этой неделе фьючерсы на кофе сорта арабика поднимались до максимума почти за 50 лет. Цены на кофе растут на фоне опасений из-за урожая 2025 года в Бразилии, где 